In [2]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm_notebook

In [7]:
folder_list[28:]

['속초 +강원도',
 '속초해수욕장 +강원도',
 '수산 +강원도',
 '수산 +양양',
 '수산항 +양양',
 '양양 +강원도',
 '오산 +강원도',
 '오산 +양양',
 '오산해수욕장 +양양',
 '인구 +강원도',
 '인구 +양양',
 '인구리 +강원도',
 '전진리 +강원도',
 '죽도 +양양',
 '하광정리 +강원도',
 '하조대 +강원도',
 '하조대 +양양',
 '하조대해수욕장 +양양',
 '후진항 +양양']

In [8]:
#각 폴더별로 자동화 
path = 'D:/Python/블로그크롤링/output/크롤링/'
folder_list = os.listdir(path)
folder_list.remove('통합')
for folder in tqdm_notebook(folder_list,desc = '전체'):
    file_list = os.listdir(path+folder)
    file_path = path+folder+'/'
    all_df = pd.DataFrame()
    for file in tqdm_notebook(file_list,desc = folder):
        read_file = pd.read_csv(file_path + file)
        all_df = pd.concat([all_df,read_file])
    
    #중복제거
    print('중복제거전row:',all_df.shape[0])
    all_df = all_df.drop_duplicates('full_text')
    final_len = all_df.shape[0]
    print('중복제거후row:',all_df.shape[0])

    #연도 달 추가
    all_df['year'] = all_df.post_dates.apply(lambda dates : int(dates.split("-")[0]))
    all_df['month'] = all_df.post_dates.apply(lambda dates : int(dates.split("-")[1]))
    all_df = all_df[['post_dates','year', 'month', 'title', 'full_text', 'url']]
    #저장 
    all_df.to_csv(f'./output/크롤링/통합/{folder}_통합_{final_len}.csv',index = False)
    print(folder,"완료")

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':



중복제거전row: 83524
중복제거후row: 81127
속초 +강원도 완료



중복제거전row: 37495
중복제거후row: 36683
속초해수욕장 +강원도 완료



중복제거전row: 38931
중복제거후row: 37302
수산 +강원도 완료


...


중복제거전row: 1083
중복제거후row: 1056
하광정리 +강원도 완료



중복제거전row: 17240
중복제거후row: 16755
하조대 +강원도 완료



중복제거전row: 21078
중복제거후row: 20399
하조대 +양양 완료



중복제거전row: 9595
중복제거후row: 9383
하조대해수욕장 +양양 완료



중복제거전row: 1381
중복제거후row: 1354
후진항 +양양 완료



### 결과 폴더 확인

In [24]:
file_list = os.listdir('./output/크롤링/통합/')

In [38]:
names = pd.Series(file_list).apply(lambda x : str.strip(x.split('_')[0].split('+')[0]))
counted = pd.Series(file_list).apply(lambda x : int(x.split('_')[2].split('.')[0]))

In [39]:
data_set = pd.DataFrame({"names":names, "counted":counted})

In [48]:
data_set

,names,counted
0,강릉,81699
1,강원도,85670
2,경포해수욕장,8739
3,광진,7973
4,광진,5044
5,광진리,666
6,기사문,1729
7,기사문,2572
8,낙산,21968
9,낙산,25325


In [47]:
data_set.groupby('names')['counted'].agg(sum).reset_index()

,names,counted
0,강릉,81699
1,강원도,85670
2,경포해수욕장,8739
3,광진,13017
4,광진리,666
5,기사문,4301
6,낙산,47293
7,낙산해수욕장,13972
8,남애,6582
9,동산,24030
